# Test nombre lignes siretise

Objective(s)

*   Lors de l’US 6 Union et intersection, nous avons compté séparément le nombre de lignes siretisés avec les tests unitaires, indépendamment des autres tests. Le niveau de matching avec les tests unitaires va de 39% a 70%. Dans cette US, nous allons faire le test avec une prise en compte de l’ensemble des tests, autrement dit, compter le nombre de lignes sirétisé lorsque les tests sont co-dépendant.
  * L’objectif principal étant de fournir un pourcentage avec le nombre de lignes siretisé.
  * Les tests vont suivre l’arborescence suivante:
    * Calcul des cas 1 à 7
      * Calcul des tests 
      * Calcul des duplicates sur index & séquence
        * Filtrer les lignes sans duplicate et tests (a definir les règles sur les tests)

## Metadata

* Metadata parameters are available here: Ressources_suDYJ#_luZqd
* Task type:
  * Jupyter Notebook
* Users: :
  * Thomas Pernet
* Watchers:
  * Thomas Pernet
* Estimated Log points:
  * One being a simple task, 15 a very difficult one
  *  9
* Task tag
  *  #sql-query,#matching,#test-codependance,#siretisation
* Toggl Tag
  * #data-analysis
* Instance [AWS/GCP]
  *   
  
## Input Cloud Storage [AWS/GCP]

If link from the internet, save it to the cloud first

### Tables [AWS/BigQuery]

1. Batch 1:
  * Select Provider: Athena
  * Select table(s): ets_insee_inpi
    * Select only tables created from the same notebook, else copy/paste selection to add new input tables
    * If table(s) does not exist, add them: Add New Table
    * Information:
      * Region: 
        * Name: Europe (Paris)
        * Code: eu-west-3
      * Database: inpi
      * Notebook construction file: https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/01_preparation/03_ETS_add_variables.md
    
## Destination Output/Delivery

* AWS
  1. Athena: 
      * Region: Europe (Paris)
      * Database: inpi
      * Tables (Add name new table): ets_inpi_insee_cases
      * List new tables
      * ets_inpi_insee_cases
  
## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)


## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import os, shutil
bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata', verbose = False) 
athena = service_athena.connect_athena(client = client,
                      bucket = 'calfdata') 

# Creation table analyse


In [2]:
query = """
/*match insee inpi 7 cas de figs*/
CREATE TABLE inpi.ets_inpi_insee_cases
WITH (
  format='PARQUET'
) AS
WITH test_proba AS (
  SELECT 
  count_initial_insee, 
    index_id, 
    sequence_id, 
    siren, 
    siret, 
    Coalesce(
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d'
        )
      ), 
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss.SSS'
        )
      ), 
      try(
        date_parse(
          datecreationetablissement, '%Y-%m-%d %hh:%mm:%ss'
        )
      ), 
      try(
        cast(
          datecreationetablissement as timestamp
        )
      )
    ) as datecreationetablissement, 
    Coalesce(
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d'
        )
      ), 
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d %hh:%mm:%ss.SSS'
        )
      ), 
      try(
        date_parse(
          "date_début_activité", '%Y-%m-%d %hh:%mm:%ss'
        )
      ), 
      try(
        cast(
          "date_début_activité" as timestamp
        )
      )
    ) as date_debut_activite, 
    etatadministratifetablissement, 
    status_admin, 
    etablissementsiege, 
    status_ets, 
    codecommuneetablissement, 
    code_commune, 
    codepostaletablissement, 
    code_postal_matching, 
    numerovoieetablissement, 
    numero_voie_matching, 
    typevoieetablissement, 
    type_voie_matching, 
    adresse_distance_inpi, 
    adresse_distance_insee, 
    list_numero_voie_matching_inpi, 
    list_numero_voie_matching_insee, 
    array_distinct(
      split(adresse_distance_inpi, ' ')
    ) as list_inpi, 
    cardinality(
      array_distinct(
        split(adresse_distance_inpi, ' ')
      )
    ) as lenght_list_inpi, 
    array_distinct(
      split(adresse_distance_insee, ' ')
    ) as list_insee, 
    cardinality(
      array_distinct(
        split(adresse_distance_insee, ' ')
      )
    ) as lenght_list_insee, 
    array_distinct(
      array_except(
        split(adresse_distance_insee, ' '), 
        split(adresse_distance_inpi, ' ')
      )
    ) as insee_except, 
    array_distinct(
      array_except(
        split(adresse_distance_inpi, ' '), 
        split(adresse_distance_insee, ' ')
      )
    ) as inpi_except, 
    CAST(
      cardinality(
        array_distinct(
          array_intersect(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            split(adresse_distance_inpi, ' '), 
            split(adresse_distance_insee, ' ')
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_,
  CAST(
      cardinality(
        array_distinct(
          array_intersect(
            list_numero_voie_matching_inpi,
            list_numero_voie_matching_insee
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection_numero_voie,
  CAST(
      cardinality(
        array_distinct(
          array_union(
            list_numero_voie_matching_inpi, 
            list_numero_voie_matching_insee
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_numero_voie,
  enseigne, enseigne1etablissement, enseigne2etablissement, enseigne3etablissement, 
  array_remove(
array_distinct(
SPLIT(
  concat(
  enseigne1etablissement,',', enseigne2etablissement,',', enseigne3etablissement),
  ',')
  ), ''
  ) as test, 
  
contains( 
         array_remove(
array_distinct(
SPLIT(
  concat(
  enseigne1etablissement,',', enseigne2etablissement,',', enseigne3etablissement),
  ',')
  ), ''
  ),enseigne
         ) AS temp_test_enseigne
  FROM 
    "inpi"."ets_insee_inpi" -- limit 10
    ) 
SELECT 
count_initial_insee,
  index_id, 
  sequence_id, 
  siren, 
  siret, 
  CASE WHEN cardinality(list_numero_voie_matching_inpi) = 0 THEN NULL ELSE list_numero_voie_matching_inpi END as list_numero_voie_matching_inpi, 
  CASE WHEN cardinality(list_numero_voie_matching_insee) = 0 THEN NULL ELSE list_numero_voie_matching_insee END as list_numero_voie_matching_insee,
  intersection_numero_voie,
  union_numero_voie,
  CASE WHEN intersection_numero_voie = union_numero_voie AND (intersection_numero_voie IS NOT NULL OR union_numero_voie IS NOT NULL) THEN 'True' 
  WHEN (intersection_numero_voie IS NULL OR union_numero_voie IS NULL) THEN 'Null'
  ELSE 'False' END AS test_list_num_voie,
  
  datecreationetablissement, 
  date_debut_activite, 
  CASE WHEN datecreationetablissement = date_debut_activite THEN 'True' WHEN datecreationetablissement IS NULL 
  OR date_debut_activite IS NULL THEN 'NULL' ELSE 'False' END AS test_date, 
  etatadministratifetablissement, 
  status_admin, 
  CASE WHEN etatadministratifetablissement = status_admin THEN 'True' WHEN etatadministratifetablissement = '' 
  OR status_admin = '' THEN 'NULL' ELSE 'False' END AS test_status_admin, 
  etablissementsiege, 
  status_ets, 
  CASE WHEN etablissementsiege = status_ets THEN 'True' WHEN etablissementsiege = '' 
  OR status_ets = '' THEN 'NULL' ELSE 'False' END AS test_siege, 
  codecommuneetablissement, 
  code_commune, 
  CASE WHEN codecommuneetablissement = code_commune THEN 'True' WHEN codecommuneetablissement = '' 
  OR code_commune = '' THEN 'NULL' ELSE 'False' END AS test_code_commune, 
  codepostaletablissement, 
  code_postal_matching, 
  numerovoieetablissement, 
  numero_voie_matching, 
  CASE WHEN numerovoieetablissement = numero_voie_matching THEN 'True' WHEN numerovoieetablissement = '' 
  OR numero_voie_matching = '' THEN 'NULL' ELSE 'False' END AS test_numero_voie, 
  typevoieetablissement, 
  type_voie_matching, 
  CASE WHEN typevoieetablissement = type_voie_matching THEN 'True' WHEN typevoieetablissement = '' 
  OR type_voie_matching = '' THEN 'NULL' ELSE 'False' END AS test_type_voie, 
  CASE WHEN cardinality(list_inpi) = 0 THEN NULL ELSE list_inpi END as list_inpi,
  
  lenght_list_inpi, 
  CASE WHEN cardinality(list_insee) = 0 THEN NULL ELSE list_insee END as list_insee,
  lenght_list_insee, 
  CASE WHEN cardinality(inpi_except) = 0 THEN NULL ELSE inpi_except END as inpi_except,
  CASE WHEN cardinality(insee_except) = 0 THEN NULL ELSE insee_except END as insee_except,
   
  intersection, 
  union_, 
  CASE WHEN intersection = union_  THEN 'CAS_1' WHEN intersection = 0 THEN 'CAS_2' WHEN lenght_list_inpi = intersection 
  AND intersection != union_ THEN 'CAS_3' WHEN lenght_list_insee = intersection 
  AND intersection != union_ THEN 'CAS_4' WHEN cardinality(insee_except) = cardinality(inpi_except) 
  AND intersection != 0 
  AND cardinality(insee_except) > 0 THEN 'CAS_5' WHEN cardinality(insee_except) > cardinality(inpi_except) 
  AND intersection != 0 
  AND cardinality(insee_except) > 0 
  AND cardinality(inpi_except) > 0 THEN 'CAS_6' WHEN cardinality(insee_except) < cardinality(inpi_except) 
  AND intersection != 0 
  AND cardinality(insee_except) > 0 
  AND cardinality(inpi_except) > 0 THEN 'CAS_7' ELSE 'CAS_NO_ADRESSE' END AS status_cas,
  enseigne, enseigne1etablissement, enseigne2etablissement, enseigne3etablissement, 
  CASE WHEN cardinality(test) = 0 THEN 'Null'
WHEN enseigne = '' THEN 'Null'
WHEN temp_test_enseigne = TRUE THEN 'True'
ELSE 'False' END AS test_enseigne 
  
FROM 
  test_proba
"""

# Analyse

## Nombre observations par cas

Le nombre d'observations doit correspondre au suivant:

|   Cas de figure | Titre                   |   Total |   Total cumulé |   pourcentage |   Pourcentage cumulé | Comment                 |
|----------------:|:------------------------|--------:|---------------:|--------------:|---------------------:|:------------------------|
|               1 | similarité parfaite     | 7775392 |        7775392 |     0.670261  |             0.670261 | Match parfait           |
|               2 | Exclusion parfaite      |  974444 |        8749836 |     0.0839998 |             0.75426  | Exclusion parfaite      |
|               3 | Match partiel parfait   |  407404 |        9157240 |     0.0351194 |             0.78938  | Match partiel parfait   |
|               4 | Match partiel parfait   |  558992 |        9716232 |     0.0481867 |             0.837566 | Match partiel parfait   |
|               5 | Match partiel compliqué | 1056406 |       10772638 |     0.0910652 |             0.928632 | Match partiel compliqué |
|               6 | Match partiel compliqué |  361242 |       11133880 |     0.0311401 |             0.959772 | Match partiel compliqué |
|               7 | Match partiel compliqué |  466671 |       11600551 |     0.0402283 |             1        | Match partiel compliqué |

In [3]:
query = """
SELECT status_cas, COUNT(*) as count
FROM ets_inpi_insee_cases 
GROUP BY status_cas
"""

## Nombre ets par cas

In [4]:
query = """
SELECT status_cas, COUNT(DISTINCT(index_id)) as distinct_ets
FROM ets_inpi_insee_cases 
GROUP BY status_cas
ORDER BY status_cas
"""

## Nombre etb unique INSEE par cas

In [5]:
query = """
SELECT * 
FROM (
SELECT status_cas, count_initial_insee, COUNT(*) as count
FROM ets_inpi_insee_cases 
GROUP BY status_cas, count_initial_insee
  )
  WHERE count_initial_insee = 1
ORDER BY status_cas, count_initial_insee
"""

## Distribution somme enseigne

In [6]:
query = """
SELECT 
  approx_percentile(sum_enseigne, ARRAY[0.25,0.50,0.75,.80,.85,.86,.87, .88, .89,.90,0.95, 0.99]) as sum_enseigne
FROM 
  ets_inpi_insee_cases 
"""

# Create table par cas

## Creation functions

La fonction ci dessous va générer le tableau d'analayse via une query, et retourne un dataframe Pandas, tout en stockant le resultat dans le dossier suivant:

- [calfdata/Analyse_cas_similarite_adresse](https://s3.console.aws.amazon.com/s3/buckets/calfdata/Analyse_cas_similarite_adresse/?region=eu-west-3&tab=overview)

In [7]:
df_ = (pd.DataFrame(data = {'index_unique': range(1,21)})
       .to_csv('index_20.csv', index = False)
      )

s3.upload_file(file_to_upload = 'index_20.csv',
            destination_in_s3 = 'TEMP_ANALYSE_SIRETISATION/INDEX_20')

In [8]:
create_table = """
CREATE EXTERNAL TABLE IF NOT EXISTS inpi.index_20 (
`index_unique`                     integer
    )
     ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"'
   )
     LOCATION 's3://calfdata/TEMP_ANALYSE_SIRETISATION/INDEX_20'
     TBLPROPERTIES ('has_encrypted_data'='false',
              'skip.header.line.count'='1');"""
output = athena.run_query(
        query=create_table,
        database='inpi',
        s3_output='INPI/sql_output'
    )

Execution ID: 42f7b3a3-1b3e-4350-862a-17bd87ffc05b


In [9]:
def create_table_test_not_false(cas = "CAS_1"):
    """
    
    """
    top = """
    SELECT count_test_list_num_voie.status_cas,
    index_unique,
    count_cas,
    test_list_num_voie,
    test_siege,
    test_enseigne,
    test_date, 
    status_admin,
    test_code_commune,
    test_type_voie
    FROM index_20 
    
    LEFT JOIN (
    SELECT count_, COUNT(count_) as count_cas
    FROM (
    SELECT COUNT(index_id) as count_
    FROM ets_inpi_insee_cases 
    WHERE status_cas = '{0}'
    GROUP BY index_id
    ORDER BY count_ DESC
  )
  GROUP BY count_
  ORDER BY count_
  ) AS count_unique
  ON index_20.index_unique = count_unique.count_ 
    """.format(cas)
    query = """
    LEFT JOIN (
    SELECT status_cas,count_index,  count(count_index) AS {0}
    FROM (
    SELECT status_cas, index_id, COUNT(test_enseigne) as count_index
    FROM ets_inpi_insee_cases 
    WHERE status_cas = '{1}' AND  {0} != 'False'
    GROUP BY status_cas, index_id
      ) as c
      GROUP BY status_cas, count_index
      ORDER BY count_index
      ) AS count_{0}
      ON index_20.index_unique = count_{0}.count_index 
    """

    for i, table in enumerate(["test_list_num_voie",
              "test_siege",
              "test_enseigne",
              "test_date", "status_admin", "test_code_commune", "test_type_voie"]):

        top += query.format(table, cas)
    top += "ORDER BY index_unique" 
    
    ### run query
    output = athena.run_query(
        query=top,
        database='inpi',
        s3_output='INPI/sql_output'
    )

    results = False
    filename = 'table_{}_test_not_false.csv'.format(cas)
    
    while results != True:
        source_key = "{}/{}.csv".format(
                            'INPI/sql_output',
                            output['QueryExecutionId']
                                   )
        destination_key = "{}/{}".format(
                                'ANALYSE_PRE_SIRETISATION',
                                filename
                            )
        
        results = s3.copy_object_s3(
                                source_key = source_key,
                                destination_key = destination_key,
                                remove = True
                            )
        
    #filename = 'table_{}_test_not_false.csv'.format('CAS_1')
    index_unique_inpi = 10981811
    reindex= ['status_cas', 'index_unique','count_cas',
              'test_list_num_voie',
              'count_num_voie',
              'test_siege',
              'count_siege',
           'test_enseigne',
               'count_enseigne',
              'test_date',
               'count_date',
              'status_admin',
              'count_admin',
              'test_code_commune',
              'count_code_commune',
           'test_type_voie',
              'count_type_voie']
    test_1 = (s3.read_df_from_s3(
            key = 'ANALYSE_PRE_SIRETISATION/{}'.format(filename), sep = ',')
     .assign(

         count_num_voie = lambda x: x['test_list_num_voie'] /  index_unique_inpi,
         count_siege = lambda x: x['test_siege'] /  index_unique_inpi,
         count_enseigne	 = lambda x: x['test_enseigne'] /  index_unique_inpi,
         count_date = lambda x: x['test_date'] /  index_unique_inpi,
         count_admin = lambda x: x['status_admin'] /  index_unique_inpi,
         count_code_commune = lambda x: x['test_code_commune'] /  index_unique_inpi,
         count_type_voie = lambda x: x['test_type_voie'] /  index_unique_inpi,
         status_cas = lambda x: x['status_cas'].fillna(cas)
     )
     .reindex(columns = reindex)
     .fillna(0)
                  .style
                  .format("{:,.0f}", subset =  [
                      "count_cas",
                      'test_list_num_voie',
                                                'test_siege',
                                                'test_enseigne',
                                                'test_date',
                                                'status_admin',
                                                'test_code_commune',
                                                'test_type_voie'])
                  .format("{:.2%}", subset =  ['count_num_voie',
                                               'count_siege',
                                               'count_enseigne',
                                               'count_date',
                                               'count_admin',
                                               'count_code_commune',
                                               'count_type_voie'])
                  .bar(subset= ['count_num_voie',
                                               'count_siege',
                                               'count_enseigne',
                                               'count_date',
                                               'count_admin',
                                               'count_code_commune',
                                               'count_type_voie'],
                       color='#d65f5f')
     )
    
    return test_1

In [10]:
create_table_test_not_false(cas = "CAS_1")

Execution ID: 0137df0b-e981-48bf-8a10-86531e08c727


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_1,1,"7,443,607","7,471,838",68.04%,"4,500,976",40.99%,"7,374,497",67.15%,"6,129,740",55.82%,"7,443,607",67.78%,"7,440,519",67.75%,"7,426,342",67.62%
1,CAS_1,2,"128,821","37,862",0.34%,"27,940",0.25%,"119,679",1.09%,"46,935",0.43%,"128,821",1.17%,"128,784",1.17%,"128,492",1.17%
2,CAS_1,3,"8,152",862,0.01%,"3,205",0.03%,"6,361",0.06%,"2,134",0.02%,"8,152",0.07%,"8,152",0.07%,"8,140",0.07%
3,CAS_1,4,"1,626",161,0.00%,931,0.01%,"1,166",0.01%,511,0.00%,"1,626",0.01%,"1,626",0.01%,"1,621",0.01%
4,CAS_1,5,576,45,0.00%,378,0.00%,317,0.00%,154,0.00%,576,0.01%,576,0.01%,576,0.01%
5,CAS_1,6,292,33,0.00%,198,0.00%,221,0.00%,140,0.00%,292,0.00%,292,0.00%,292,0.00%
6,CAS_1,7,108,8,0.00%,74,0.00%,69,0.00%,39,0.00%,108,0.00%,108,0.00%,108,0.00%
7,CAS_1,8,110,7,0.00%,100,0.00%,93,0.00%,65,0.00%,110,0.00%,110,0.00%,110,0.00%
8,CAS_1,9,66,2,0.00%,73,0.00%,51,0.00%,27,0.00%,66,0.00%,66,0.00%,66,0.00%
9,CAS_1,10,95,23,0.00%,84,0.00%,61,0.00%,61,0.00%,95,0.00%,95,0.00%,95,0.00%


In [11]:
create_table_test_not_false(cas = "CAS_2")

Execution ID: 6777e09c-5913-43ee-adce-4feb5fc50700


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_2,1,"649,128","263,553",2.40%,"342,898",3.12%,"634,377",5.78%,"395,347",3.60%,"649,128",5.91%,"647,955",5.90%,"135,749",1.24%
1,CAS_2,2,"74,413","21,537",0.20%,"37,218",0.34%,"68,579",0.62%,"19,909",0.18%,"74,413",0.68%,"74,318",0.68%,"6,556",0.06%
2,CAS_2,3,"16,586","4,352",0.04%,"10,196",0.09%,"14,372",0.13%,"4,239",0.04%,"16,586",0.15%,"16,570",0.15%,"1,019",0.01%
3,CAS_2,4,"5,976","1,384",0.01%,"4,470",0.04%,"4,905",0.04%,"1,681",0.02%,"5,976",0.05%,"5,970",0.05%,396,0.00%
4,CAS_2,5,"3,002",649,0.01%,"2,489",0.02%,"2,495",0.02%,870,0.01%,"3,002",0.03%,"2,994",0.03%,191,0.00%
5,CAS_2,6,"1,866",370,0.00%,"1,696",0.02%,"1,610",0.01%,563,0.01%,"1,866",0.02%,"1,862",0.02%,74,0.00%
6,CAS_2,7,"1,310",239,0.00%,"1,191",0.01%,"1,118",0.01%,373,0.00%,"1,310",0.01%,"1,310",0.01%,74,0.00%
7,CAS_2,8,966,161,0.00%,923,0.01%,801,0.01%,274,0.00%,966,0.01%,966,0.01%,59,0.00%
8,CAS_2,9,803,129,0.00%,728,0.01%,678,0.01%,236,0.00%,803,0.01%,801,0.01%,63,0.00%
9,CAS_2,10,554,103,0.00%,544,0.00%,450,0.00%,184,0.00%,554,0.01%,554,0.01%,34,0.00%


In [12]:
create_table_test_not_false(cas = "CAS_3")

Execution ID: 6a427562-e4f5-450c-bf3f-acb2aad0c22a


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_3,1,"387,589","333,616",3.04%,"234,578",2.14%,"381,978",3.48%,"295,610",2.69%,"387,589",3.53%,"387,291",3.53%,"310,390",2.83%
1,CAS_3,2,"6,915","2,957",0.03%,"2,621",0.02%,"5,983",0.05%,"2,257",0.02%,"6,915",0.06%,"6,913",0.06%,"5,743",0.05%
2,CAS_3,3,721,266,0.00%,402,0.00%,569,0.01%,196,0.00%,721,0.01%,720,0.01%,577,0.01%
3,CAS_3,4,246,84,0.00%,161,0.00%,161,0.00%,67,0.00%,246,0.00%,246,0.00%,210,0.00%
4,CAS_3,5,72,25,0.00%,62,0.00%,54,0.00%,22,0.00%,72,0.00%,72,0.00%,62,0.00%
5,CAS_3,6,69,12,0.00%,42,0.00%,50,0.00%,15,0.00%,69,0.00%,69,0.00%,69,0.00%
6,CAS_3,7,9,2,0.00%,8,0.00%,6,0.00%,1,0.00%,9,0.00%,9,0.00%,8,0.00%
7,CAS_3,8,28,21,0.00%,25,0.00%,9,0.00%,11,0.00%,28,0.00%,28,0.00%,26,0.00%
8,CAS_3,9,13,5,0.00%,6,0.00%,10,0.00%,3,0.00%,13,0.00%,13,0.00%,12,0.00%
9,CAS_3,10,13,1,0.00%,14,0.00%,5,0.00%,5,0.00%,13,0.00%,13,0.00%,13,0.00%


In [13]:
create_table_test_not_false(cas = "CAS_4")

Execution ID: 8b4b41af-1df7-4313-87b0-4c25310ae2d7


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_4,1,"525,832","463,298",4.22%,"331,856",3.02%,"519,393",4.73%,"419,766",3.82%,"525,832",4.79%,"523,180",4.76%,"516,382",4.70%
1,CAS_4,2,"10,572","4,106",0.04%,"3,154",0.03%,"9,468",0.09%,"3,133",0.03%,"10,572",0.10%,"10,529",0.10%,"9,679",0.09%
2,CAS_4,3,830,193,0.00%,491,0.00%,625,0.01%,182,0.00%,830,0.01%,827,0.01%,791,0.01%
3,CAS_4,4,236,51,0.00%,140,0.00%,160,0.00%,55,0.00%,236,0.00%,235,0.00%,218,0.00%
4,CAS_4,5,61,15,0.00%,58,0.00%,41,0.00%,17,0.00%,61,0.00%,61,0.00%,55,0.00%
5,CAS_4,6,43,7,0.00%,31,0.00%,30,0.00%,10,0.00%,43,0.00%,43,0.00%,41,0.00%
6,CAS_4,7,39,15,0.00%,34,0.00%,9,0.00%,10,0.00%,39,0.00%,39,0.00%,39,0.00%
7,CAS_4,8,28,19,0.00%,29,0.00%,13,0.00%,20,0.00%,28,0.00%,28,0.00%,28,0.00%
8,CAS_4,9,15,2,0.00%,18,0.00%,11,0.00%,11,0.00%,15,0.00%,15,0.00%,14,0.00%
9,CAS_4,10,55,29,0.00%,49,0.00%,39,0.00%,14,0.00%,55,0.00%,55,0.00%,58,0.00%


In [14]:
create_table_test_not_false(cas = "CAS_5")

Execution ID: fb3a4695-3c99-4355-b47f-1063d20e3c18


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_5,1,"943,789","788,667",7.18%,"561,655",5.11%,"929,050",8.46%,"718,702",6.54%,"943,789",8.59%,"942,868",8.59%,"724,028",6.59%
1,CAS_5,2,"33,429","6,882",0.06%,"13,699",0.12%,"30,388",0.28%,"10,529",0.10%,"33,429",0.30%,"33,413",0.30%,"27,847",0.25%
2,CAS_5,3,"4,467",664,0.01%,"3,026",0.03%,"3,627",0.03%,"1,267",0.01%,"4,467",0.04%,"4,466",0.04%,"3,883",0.04%
3,CAS_5,4,"1,474",218,0.00%,"1,220",0.01%,"1,145",0.01%,484,0.00%,"1,474",0.01%,"1,474",0.01%,"1,319",0.01%
4,CAS_5,5,741,100,0.00%,699,0.01%,595,0.01%,261,0.00%,741,0.01%,741,0.01%,657,0.01%
5,CAS_5,6,470,52,0.00%,410,0.00%,368,0.00%,192,0.00%,470,0.00%,470,0.00%,453,0.00%
6,CAS_5,7,203,34,0.00%,182,0.00%,168,0.00%,103,0.00%,203,0.00%,203,0.00%,209,0.00%
7,CAS_5,8,134,42,0.00%,140,0.00%,108,0.00%,64,0.00%,134,0.00%,134,0.00%,141,0.00%
8,CAS_5,9,115,29,0.00%,116,0.00%,82,0.00%,49,0.00%,115,0.00%,115,0.00%,89,0.00%
9,CAS_5,10,124,34,0.00%,113,0.00%,105,0.00%,55,0.00%,124,0.00%,124,0.00%,122,0.00%


In [15]:
create_table_test_not_false(cas = "CAS_6")

Execution ID: c8beef53-2a24-4194-a556-fba01fd5320a


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_6,1,"296,702","165,602",1.51%,"175,751",1.60%,"288,908",2.63%,"192,139",1.75%,"296,702",2.70%,"296,322",2.70%,"215,807",1.97%
1,CAS_6,2,"16,116","3,783",0.03%,"9,376",0.09%,"14,014",0.13%,"4,616",0.04%,"16,116",0.15%,"16,105",0.15%,"13,114",0.12%
2,CAS_6,3,"3,087",576,0.01%,"2,369",0.02%,"2,352",0.02%,797,0.01%,"3,087",0.03%,"3,086",0.03%,"2,622",0.02%
3,CAS_6,4,"1,193",232,0.00%,"1,037",0.01%,902,0.01%,350,0.00%,"1,193",0.01%,"1,193",0.01%,"1,029",0.01%
4,CAS_6,5,615,151,0.00%,559,0.01%,485,0.00%,167,0.00%,615,0.01%,613,0.01%,535,0.00%
5,CAS_6,6,376,113,0.00%,346,0.00%,278,0.00%,96,0.00%,376,0.00%,375,0.00%,298,0.00%
6,CAS_6,7,264,46,0.00%,263,0.00%,235,0.00%,107,0.00%,264,0.00%,264,0.00%,235,0.00%
7,CAS_6,8,167,37,0.00%,147,0.00%,121,0.00%,52,0.00%,167,0.00%,167,0.00%,119,0.00%
8,CAS_6,9,84,19,0.00%,82,0.00%,48,0.00%,19,0.00%,84,0.00%,84,0.00%,49,0.00%
9,CAS_6,10,78,30,0.00%,78,0.00%,53,0.00%,64,0.00%,78,0.00%,78,0.00%,66,0.00%


In [16]:
create_table_test_not_false(cas = "CAS_7")

Execution ID: 1eab8981-3deb-4c7f-b266-bc6679e6a392


,status_cas,index_unique,count_cas,test_list_num_voie,count_num_voie,test_siege,count_siege,test_enseigne,count_enseigne,test_date,count_date,status_admin,count_admin,test_code_commune,count_code_commune,test_type_voie,count_type_voie
0,CAS_7,1,"367,508","234,930",2.14%,"224,637",2.05%,"360,218",3.28%,"258,847",2.36%,"367,508",3.35%,"366,478",3.34%,"292,501",2.66%
1,CAS_7,2,"25,116","6,757",0.06%,"11,670",0.11%,"22,960",0.21%,"7,493",0.07%,"25,116",0.23%,"25,070",0.23%,"20,543",0.19%
2,CAS_7,3,"4,395",863,0.01%,"2,889",0.03%,"3,576",0.03%,"1,201",0.01%,"4,395",0.04%,"4,389",0.04%,"3,735",0.03%
3,CAS_7,4,"1,632",414,0.00%,"1,358",0.01%,"1,322",0.01%,476,0.00%,"1,632",0.01%,"1,627",0.01%,"1,355",0.01%
4,CAS_7,5,824,198,0.00%,704,0.01%,643,0.01%,246,0.00%,824,0.01%,822,0.01%,689,0.01%
5,CAS_7,6,498,139,0.00%,476,0.00%,377,0.00%,182,0.00%,498,0.00%,498,0.00%,408,0.00%
6,CAS_7,7,377,70,0.00%,342,0.00%,266,0.00%,82,0.00%,377,0.00%,376,0.00%,320,0.00%
7,CAS_7,8,217,57,0.00%,201,0.00%,176,0.00%,68,0.00%,217,0.00%,217,0.00%,206,0.00%
8,CAS_7,9,155,54,0.00%,156,0.00%,120,0.00%,34,0.00%,155,0.00%,155,0.00%,126,0.00%
9,CAS_7,10,130,36,0.00%,125,0.00%,101,0.00%,72,0.00%,130,0.00%,130,0.00%,121,0.00%


# Generation report

In [17]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [18]:
def create_report(extension = "html"):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [19]:
create_report(extension = "html")

Report Available at this adress:
 C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching\Notebooks_matching\Data_preprocessed\programme_matching\02_siretisation\Reports\07_pourcentage_siretisation_v3.html
